In [2]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.201-b09, mixed mode)
  Starting server from C:\Users\tawde\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\tawde\AppData\Local\Temp\tmpd12sfw3y
  JVM stdout: C:\Users\tawde\AppData\Local\Temp\tmpd12sfw3y\h2o_tawde_started_from_python.out
  JVM stderr: C:\Users\tawde\AppData\Local\Temp\tmpd12sfw3y\h2o_tawde_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.2
H2O cluster version age:,8 days
H2O cluster name:,H2O_from_python_tawde_c9tr6f
H2O cluster total nodes:,1
H2O cluster free memory:,3.523 Gb
H2O cluster total cores:,0
H2O cluster allowed cores:,0
H2O cluster status:,"accepting new members, healthy"


In [17]:
import numpy as np

def MAPE(test, predict):
    mape = 0

    for i, j in zip(test['App'].as_data_frame().values, predict.as_data_frame().values):
        mape += np.abs((i-j)/i)

    mape = (mape/test.shape[0])*100
    accuracy = 100 - np.mean(mape)
    print('Accuracy:', round(accuracy, 2), '%.')

    return mape

In [4]:
df = h2o.import_file('../Final Project/Data/model_input.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
df.shape

(16682, 9)

In [6]:
train, test = df.split_frame(
    ratios=[0.8], 
    seed=1234, 
    destination_frames=['train.hex', 'test.hex']
)

In [7]:
print('train shape: {}'.format(train.shape))
print('test shape: {}'.format(test.shape))

train shape: (13417, 9)
test shape: (3265, 9)


In [8]:
y = 'App'

In [9]:
predictors = ['Good Cost','Bad Cost','Good Recommendation','Bad Recommendation','Good Stability','Bad Stability','Ads','No Ads']

In [12]:
aml = H2OAutoML(max_runtime_secs= 600, seed = 1, project_name = "lending_club")

In [13]:
aml.train(x= predictors, y= y, training_frame= train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [14]:
aml.leaderboard

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_5_AutoML_20190425_141959,0.93218,0.965495,0.93218,0.793557,0.308814
StackedEnsemble_BestOfFamily_AutoML_20190425_141959,0.933103,0.965972,0.933103,0.792817,0.308934
GBM_grid_1_AutoML_20190425_141959_model_6,0.933335,0.966093,0.933335,0.793376,0.308965
StackedEnsemble_AllModels_AutoML_20190425_141959,0.933505,0.966181,0.933505,0.793224,0.30897
GBM_2_AutoML_20190425_141959,0.934694,0.966796,0.934694,0.792798,0.309118
GBM_3_AutoML_20190425_141959,0.935137,0.967025,0.935137,0.792749,0.309167
GBM_4_AutoML_20190425_141959,0.935601,0.967265,0.935601,0.792725,0.309224
GBM_grid_1_AutoML_20190425_141959_model_8,0.936345,0.967649,0.936345,0.79305,0.309419
GBM_grid_1_AutoML_20190425_141959_model_4,0.936923,0.967948,0.936923,0.793092,0.309345
GBM_grid_1_AutoML_20190425_141959_model_5,0.937953,0.96848,0.937953,0.794786,0.309717


In [15]:
preds = aml.predict(test)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [18]:
MAPE(test, preds)

Accuracy: 52.49 %.


array([47.50594801])